In [ ]:
import os

from pandas import DataFrame, date_range
import numpy as np

from idealreport.settings import load_settings
from idealreport import create_html
from idealreport.reporter import Reporter
import htmltag

In [ ]:
# load settings from settings.hjson
settings = load_settings()

# instantiate reporter object and set output location
output_file = os.path.join(settings['output_path'], 'sample_plots.html')
r = Reporter(title='Sample Plots', output_file=output_file)

# report start
r.h += htmltag.h3('This report provides an overview of the different plot types.')

In [ ]:
# vertical bar chart

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                  'Stat 1': np.random.randn(5).tolist(),
                  'Stat 2': np.random.randn(5).tolist(),
                })

# note the x values are generally assumed to be in the index, with each non index column being a series of y values
# a few exceptions are outlined below
ch = ch.set_index('Entity')
ch = ch.sort_values(by='Stat 1', ascending=False)

# each object can be built by create_html for more flexibility

# graphs are generated from dictionaries with attributes and a data dictionary as an entry
bar_plot = {
    'title': 'Vertical Bar Chart by Create HTML',
    'data': [
        {
            'df': ch,
            'type': 'bar',
            'orientation': 'v',
        },
    ],
    'labelX': 'Entity',
    'labelY': '%',
    'staticPlot': False
}

r.h += htmltag.h4('Bar Charts')
r.h += create_html.plot(bar_plot)

# all of the remaining examples just use the reporter object to build the dictionaries for create_html

r.plot.bar(df=ch, title='Vertical Bar Chart by Reporter', xlabel='Entity', ylabel='%')

In [ ]:
# stacked horizontal bar chart, notice additional entry in the data field for orientation

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                'Foo': np.random.randn(5).tolist(),
                'Bar': np.random.randn(5).tolist(),
               })
ch = ch.set_index('Entity')
ch = ch[['Foo', 'Bar']] # make sure columns have desired order

# stacked bars behave oddly with mixed positive / negative numbers, so only use when all numbers are positive or negative within a series
ch = ch.abs()
ch = ch*100

r.plot.barh(df=ch, title='Horizontal Stacked Bar Chart', stacked=True, xlabel='$')

In [ ]:
# histogram, note that we don't use the index for x values here because there is no x value other than the series

ch = DataFrame({'Foo': np.random.randn(50).tolist(),
                'Bar': np.random.randn(50).tolist(),
               })

r.plot.histo(df=ch, title='Histogram by Reporter', xlabel='$')

In [ ]:
# line chart

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch.a = range(20)
ch = ch.set_index('a')

r.h += htmltag.h4('Line Charts')
r.plot.line(df=ch, title='Line Chart', xlabel='Entity', ylabel='%')

In [ ]:
# time series

ind = date_range('2017-11-02 9:00', periods=20, freq='T')
ch = DataFrame(data=np.random.randn(20,2), index=ind)
ch.columns = ['a', 'b']

# just like a regular line except the index is timestamps
r.plot.time(df=ch, title='Time Series', xlabel='Time', ylabel='%')

In [ ]:
# scatter plot

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch = ch.set_index('a')

r.h += htmltag.h4('Scatter Plots')
r.plot.scatter(df=ch, title='Scatter Plot', xlabel='alpha', ylabel='beta')

In [ ]:
# mixed plot types on one graph

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['a', 'b1']
ch1.sort_values(by='a', ascending=True, inplace=True)
ch1 = ch1.set_index('a')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['a', 'b2']
ch2.sort_values(by='a', ascending=True, inplace=True)
ch2 = ch2.set_index('a')

r.h += htmltag.h4('Mixed Type Plots')
r.plot.multi(dfs=[ch1, ch2], types=['line', 'bar'], title='Mixed Line and Bar Plot', xlabel='x label', ylabel='y label')

In [ ]:
# different axes

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['x', 'a']
ch1.x = range(20)
ch1 = ch1.set_index('x')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['x', 'b']
ch2.x = range(20)
ch2 = ch2.set_index('x')
ch2.b *= 100

# y2_axis is a list of booleans indicating whether each df should be tied to the y2 axis or not
r.plot.multi(dfs=[ch1, ch2], types=['line', 'line'], title='Lines with Different Y Axes', xlabel='x', ylabel='ya', y2_axis=[False,True], y2label='yb')

In [ ]:
# ohlc

ch = DataFrame(np.random.randn(4, 4))
ch.columns = ['open', 'high', 'low', 'close']
ch['high'] = ch['high'] + 3
ch['low'] = ch['low'] - 3

ch['dealer'] = range(4)
ch = ch.set_index('dealer')

r.h += htmltag.h4('Open High Low Close (OHCL) Plot')
r.plot.ohlc(df=ch, title='OHLC Plot by Reporter', xlabel='price', ylabel='instrument')

In [ ]:
# pie

ch = DataFrame(data=[1,2,3,4], index=['Factor %s' % x for x in range(4)])

r.h += htmltag.h4('Pie Charts')
r.plot.pie(df=ch, title='Pie')

# only difference for a donut is the hole argument
r.plot.pie(df=ch, title='Donut', hole=.4, height=300)

In [ ]:
# error bar charts

ch = DataFrame(np.random.randn(20, 4))
ch.columns = ['x','y','e1','e2']
ch = ch.set_index('x')
ch['e1'] = ch['e1'].abs()*0.1
ch['e2'] = ch['e2'].abs()*0.1

r.h += htmltag.h4('Error Bar Charts')
# note only first 3 columns are used for symmetric error bars
r.plot.errbar(df=ch, title='Symmetric Error Bars')

# just add the flag symmetric=False, otherwise it defaults to True like in the call above
r.plot.errbar(df=ch, title='Asymmetric Error Bars', symmetric=False)

In [ ]:
# error line plots

ch = DataFrame({'day': ['1', '2', '3', '4', '5', '6', '7', '8'],
                'mean': [15, 13, 7, 18, 19, 24, 25, 28],
                'stdev': [1, 2, 1.3, 1, 2.5, .5, 1.5, .75]})
ch = ch[['day', 'mean', 'stdev']] # make sure columns have desired order
ch = ch.set_index('day')

r.h += htmltag.h4('Error Line Plots')
r.plot.errline(df=ch, title='Symmetric Error Bars')

In [ ]:
# generate and save the report HTML
r.generate()